___

<a href='https://mainacademy.ua/'> <img src='https://mainacademy.ua/wp-content/uploads/2019/03/logo-main.png' alt = 'Хмм, щось з інтернетом'/></a>
___

# Module 4: Work with data professionally!

## Lab work 4



#### Мета: 

* навчитися обробляти дані
* "парсити" сайти

### Завдання 1:

#### Ознайомитися із API Національного банку 
https://bank.gov.ua/ua/open-data/api-dev



1. Підключитися до API.


In [192]:
import requests
import json
import datetime
import pprint
import math
import pickle

pp = pprint.PrettyPrinter(sort_dicts=True)

def get_exchange_rate_for_period(start, end, currency = 'usd'):
  return requests.get(f'https://bank.gov.ua/NBU_Exchange/exchange_site?start={start}&end={end}&valcode={currency}&sort=exchangedate&order=asc&json')

def get_average(data):
  result = sum(row['rate_per_unit'] for row in data) / len(data)
  return round(result, 2)

2. Отримати курс долара за останній рік.

In [152]:
date_format = '%Y%m%d'
date_formatted = '%Y-%m-%d'
current_date = datetime.datetime.now().strftime(date_format)

year_ago = datetime.datetime.now() - datetime.timedelta(days=365)
year_ago_formatted = year_ago.strftime(date_format)

usd_data = get_exchange_rate_for_period(year_ago_formatted, current_date).json()
usd_price_for_year = get_average(usd_data)

# print(json.dumps(usd_data, indent=4))
print(f'Usd average for the priod: {year_ago.strftime(date_formatted)} - {datetime.datetime.now().strftime(date_formatted)} is {usd_price_for_year}')

Usd average for the priod: 2022-03-06 - 2023-03-06 is 33.83


3. Вивести середнє значення та відхилення курсу за кожний місяць.

In [205]:
months_dict = {}
usd_average_months = {}
period = 12

def get_previous_month_date(today):
  first = today.replace(day=1)
  last_month = first - datetime.timedelta(days=1)
  return last_month

current = datetime.datetime.now()
previous = ''

for i in range(period):
  previous = get_previous_month_date(current)
  previous_start = previous.replace(day=1).strftime(date_format)
  previous_end = previous.strftime(date_format)
  start_format = previous.replace(day=1).strftime(date_formatted)
  end_format = previous.strftime(date_formatted)

  months_dict[f'{start_format} - {end_format}'] = get_exchange_rate_for_period(f'{previous_start}', f'{previous_end}').json()
  
  current = previous

average = 0

for key, value in months_dict.items():
  usd_average_months[key] = get_average(value)
  average += get_average(value)

usd_average_price = average / len(months_dict)

average_prices_sum = 0

for key, value in usd_average_months.items():
  average_prices_sum += (value - usd_average_price) ** 2

deviation = math.sqrt(average_prices_sum / len(usd_average_months))

usd_average_months.update({'deviation': round(deviation, 2)})


4. Дану інформацію записати у файл за допомогою pickle.

In [206]:
with open('./usd_price_info.pickle', 'wb') as handle:
  pickle.dump(usd_average_months, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('./usd_price_info.pickle', 'rb') as handle:
  pickle_data = pickle.load(handle)

pp.pprint(pickle_data)

{'2022-03-01 - 2022-03-31': 29.25,
 '2022-04-01 - 2022-04-30': 29.25,
 '2022-05-01 - 2022-05-31': 29.25,
 '2022-06-01 - 2022-06-30': 29.25,
 '2022-07-01 - 2022-07-31': 31.85,
 '2022-08-01 - 2022-08-31': 36.57,
 '2022-09-01 - 2022-09-30': 36.57,
 '2022-10-01 - 2022-10-31': 36.57,
 '2022-11-01 - 2022-11-30': 36.57,
 '2022-12-01 - 2022-12-31': 36.57,
 '2023-01-01 - 2023-01-31': 36.57,
 '2023-02-01 - 2023-02-28': 36.57,
 'deviation': 3.42}


### Завдання 2:

Потрібно проаналізувати всі товари на сайті: 
https://smallpacking.agrosem.ua/shop/

Створіть базу даних, що містить таблицю із такими стовбцями: 
    - артикул товару
    - найменування
    - ціна 
    - вага

In [96]:
import requests
from bs4 import BeautifulSoup
import math

shop_url = "https://smallpacking.agrosem.ua/products/"
cookies = {'PHPSESSID': 'r0te1cu8bkq4aluljh0gvnn1uu'}
shop_data = requests.get(shop_url, cookies=cookies)

soup = BeautifulSoup(shop_data.text, 'html.parser')
all_products_text = soup.find('div', class_='catalog_head').find_all('p', class_='products')

products_per_page = 12
all_products_length = [int(i) for i in all_products_text[0].text.split() if i.isdigit()][1]
pages = all_products_length / products_per_page
pages = math.ceil(pages)

In [103]:
import json
import re

products = []

def extract_weight(string):
    match = re.search(r'\d.*$', string)
    return match.group() if match else ''

def format_weight(weight):
    format = r'\d.* [кг|гр|л|мл|шт]+'
    match = re.search(format, weight)
    return match.group() if match else ''

for i in range(pages):
    parse_url = f'{shop_url}?page={i + 1}'
    shop_data = requests.get(parse_url, cookies=cookies)
    soup = BeautifulSoup(shop_data.text, 'html.parser')

    products_items = soup.find(
        'div', class_='items_wrap').find_all('div', class_='item')

    for product in products_items:
        content = product.get('content')
        content = json.loads(content)
        weight = extract_weight(content["name"])
        weight = format_weight(weight)
        products.append((content["id"], content["name"], content["price"], content["url"], weight))

In [104]:
import sqlite3

con = sqlite3.connect('./agrosem-products.db')
cur = con.cursor()

cur.execute('''
    CREATE TABLE IF NOT EXISTS products (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        product_id INTEGER,
        name TEXT,
        price REAL,
        link TEXT,
        weight TEXT
    )
''')
            
cur.executemany('''
    INSERT INTO products (product_id, name, price, link, weight) VALUES (?, ?, ?, ?, ?)
''', products)

con.commit()

con.close()


### Завдання 3: 

Ознайомтеся із роботою SQLite та відповідним модулем у Python.
Завантажте базу даних для виконання лабораторної роботи.
Підключіться до завантаженої бази SQLite.

1. Виведіть інформацію про дану базу.

In [19]:
import sqlite3
import pprint as pp

con = sqlite3.connect('./tysql.sqlite')

sql = 'SELECT * FROM sqlite_master WHERE type="table"'

cur = con.cursor()
cur.execute(sql)

pp.pprint(cur.fetchall())

[('table',
  'Customers',
  'Customers',
  2,
  'CREATE TABLE Customers\n'
  '(\n'
  '  cust_id      char(10)  NOT NULL ,\n'
  '  cust_name    char(50)  NOT NULL ,\n'
  '  cust_address char(50)  NULL ,\n'
  '  cust_city    char(50)  NULL ,\n'
  '  cust_state   char(5)   NULL ,\n'
  '  cust_zip     char(10)  NULL ,\n'
  '  cust_country char(50)  NULL ,\n'
  '  cust_contact char(50)  NULL ,\n'
  '  cust_email   char(255) NULL ,\n'
  '  PRIMARY KEY (cust_id)\n'
  ')'),
 ('table',
  'OrderItems',
  'OrderItems',
  4,
  'CREATE TABLE OrderItems\n'
  '(\n'
  '  order_num  int          NOT NULL                      ,\n'
  '  order_item int          NOT NULL                      ,\n'
  '  prod_id    char(10)     NOT NULL                      ,\n'
  '  quantity   int          NOT NULL                      ,\n'
  '  item_price decimal(8,2) NOT NULL                      ,\n'
  '  PRIMARY KEY (order_num, order_item)                   ,\n'
  '  FOREIGN KEY (order_num) REFERENCES Orders (order_num) 

2. Виведіть перелік всіх таблиць.

In [21]:
sql = 'SELECT name FROM sqlite_master WHERE type="table"'

cur.execute(sql)

pp.pprint(cur.fetchall())


[('Customers',), ('OrderItems',), ('Orders',), ('Products',), ('Vendors',)]


3. Список всіх cust_id з таблиці Customers table.

In [22]:
sql = 'SELECT cust_id FROM Customers'

cur.execute(sql)

pp.pprint(cur.fetchall())

[('1000000001',),
 ('1000000002',),
 ('1000000003',),
 ('1000000004',),
 ('1000000005',)]


4. Всю таблицю Customers table

In [23]:
sql = 'SELECT * FROM Customers'

cur.execute(sql)

pp.pprint(cur.fetchall())

[('1000000001',
  'Village Toys',
  '200 Maple Lane',
  'Detroit',
  'MI',
  '44444',
  'USA',
  'John Smith',
  'sales@villagetoys.com'),
 ('1000000002',
  'Kids Place',
  '333 South Lake Drive',
  'Columbus',
  'OH',
  '43333',
  'USA',
  'Michelle Green',
  None),
 ('1000000003',
  'Fun4All',
  '1 Sunny Place',
  'Muncie',
  'IN',
  '42222',
  'USA',
  'Jim Jones',
  'jjones@fun4all.com'),
 ('1000000004',
  'Fun4All',
  '829 Riverside Drive',
  'Phoenix',
  'AZ',
  '88888',
  'USA',
  'Denise L. Stephens',
  'dstephens@fun4all.com'),
 ('1000000005',
  'The Toy Store',
  '4545 53rd Street',
  'Chicago',
  'IL',
  '54545',
  'USA',
  'Kim Howard',
  None)]


5. Список клієнтів (cust_names) відсортованих від Z до A

In [29]:
sql = 'SELECT cust_name FROM Customers ORDER BY cust_name DESC'

cur.execute(sql)

pp.pprint(cur.fetchall())

[('Village Toys',),
 ('The Toy Store',),
 ('Kids Place',),
 ('Fun4All',),
 ('Fun4All',)]


6. Таблицю клієнтів та замовлень (cust_id і order_num) . Відсортуйте по клієнту і потім по даті замовлення

In [32]:
sql = 'SELECT * FROM Customers ORDER BY cust_id ASC'

cur.execute(sql)

pp.pprint(cur.fetchall())

sql = 'SELECT * FROM Orders ORDER BY order_num ASC'

cur.execute(sql)

pp.pprint(cur.fetchall())

[('1000000001',
  'Village Toys',
  '200 Maple Lane',
  'Detroit',
  'MI',
  '44444',
  'USA',
  'John Smith',
  'sales@villagetoys.com'),
 ('1000000002',
  'Kids Place',
  '333 South Lake Drive',
  'Columbus',
  'OH',
  '43333',
  'USA',
  'Michelle Green',
  None),
 ('1000000003',
  'Fun4All',
  '1 Sunny Place',
  'Muncie',
  'IN',
  '42222',
  'USA',
  'Jim Jones',
  'jjones@fun4all.com'),
 ('1000000004',
  'Fun4All',
  '829 Riverside Drive',
  'Phoenix',
  'AZ',
  '88888',
  'USA',
  'Denise L. Stephens',
  'dstephens@fun4all.com'),
 ('1000000005',
  'The Toy Store',
  '4545 53rd Street',
  'Chicago',
  'IL',
  '54545',
  'USA',
  'Kim Howard',
  None)]
[(20005, '2012-05-01', '1000000001'),
 (20006, '2012-01-12', '1000000003'),
 (20007, '2012-01-30', '1000000004'),
 (20008, '2012-02-03', '1000000005'),
 (20009, '2012-02-08', '1000000001')]


7. Таблицю (на основі Items) з кількість та вартістю товару. Відсортуйте в порядку спадання по кількості та вартості

In [36]:
sql = 'SELECT quantity, item_price FROM OrderItems ORDER BY quantity ASC, item_price ASC'

cur.execute(sql)

pp.pprint(cur.fetchall())

[(5, 4.99),
 (5, 11.99),
 (10, 3.49),
 (10, 3.49),
 (10, 3.49),
 (10, 8.99),
 (10, 11.99),
 (20, 5.99),
 (50, 4.49),
 (50, 11.49),
 (100, 2.99),
 (100, 2.99),
 (100, 2.99),
 (100, 5.49),
 (100, 10.99),
 (250, 2.49),
 (250, 2.49),
 (250, 2.49)]


8. Товар (з таблиці Products), ціна якого становить 9.49

In [37]:
sql = 'SELECT * FROM Products WHERE prod_price = 9.49'

cur.execute(sql)

pp.pprint(cur.fetchall())

[('RYL01',
  'FNG01',
  'King doll',
  9.49,
  '12 inch king doll with royal garments and crown'),
 ('RYL02',
  'FNG01',
  'Queen doll',
  9.49,
  '12 inch queen doll with royal garments and crown')]


9. Виведіть назву товару та ціну, яка лежить в діапазоні від  3 до 6. Відсортуйте результат в по ціні в порядку зростання

In [38]:
sql = 'SELECT prod_name, prod_price FROM Products WHERE prod_price BETWEEN 3 AND 6 ORDER BY prod_price ASC'

cur.execute(sql)

pp.pprint(cur.fetchall())

[('Fish bean bag toy', 3.49),
 ('Bird bean bag toy', 3.49),
 ('Rabbit bean bag toy', 3.49),
 ('Raggedy Ann', 4.99),
 ('8 inch teddy bear', 5.99)]


10. Кількість товару, що було продано

In [40]:
sql = 'SELECT count(*) FROM Orders'

cur.execute(sql)

pp.pprint(cur.fetchall())

[(5,)]


11. Кількість найменувань товару, ціна якого більша за 4

In [41]:
sql = 'SELECT count(prod_name) FROM Products WHERE prod_price > 4'

cur.execute(sql)

pp.pprint(cur.fetchall())

[(6,)]


12. Розробіть алгоритм для виведення 3 найдорожчих товарів в базі

In [47]:
sql = 'SELECT prod_name, prod_price FROM Products ORDER BY prod_price DESC LIMIT 3'

cur.execute(sql)

pp.pprint(cur.fetchall())

[('18 inch teddy bear', 11.99), ('King doll', 9.49), ('Queen doll', 9.49)]


13. Підрахуйте кількість замовлень для кожного клієнта використовуючи підзапити

In [52]:
sql = """
SELECT 
    cust_name, 
    (
        SELECT count(order_num)
        FROM Orders
        WHERE Orders.cust_id = Customers.cust_id
    )
    customer_orders_count
FROM 
    Customers
ORDER BY customer_orders_count DESC
"""

cur.execute(sql)

pp.pprint(cur.fetchall())

[('Village Toys', 2),
 ('Fun4All', 1),
 ('Fun4All', 1),
 ('The Toy Store', 1),
 ('Kids Place', 0)]


14. Виведіть список клієнтів та їх замовлення (використайте JOIN)

In [61]:
sql = """
SELECT
    cust_name,
    order_num
FROM
    Customers
INNER JOIN Orders
    ON Customers.cust_id = Orders.cust_id
"""

cur.execute(sql)

pp.pprint(cur.fetchall())

[('Village Toys', 20005),
 ('Fun4All', 20006),
 ('Fun4All', 20007),
 ('The Toy Store', 20008),
 ('Village Toys', 20009)]


15. Зробіть два запити: список товарів, ціна яких менша 5 та список товарів, ціна який більша рівна 5. Використайте UNION для об'єднання цих запитів

In [67]:
sql = """
SELECT
    prod_name,
    prod_price
FROM
    Products
WHERE
    prod_price < 5

UNION

SELECT
    prod_name,
    prod_price
FROM
    Products
WHERE
    prod_price >= 5

ORDER BY prod_price DESC
"""

cur.execute(sql)

pp.pprint(cur.fetchall())

con.close()

[('18 inch teddy bear', 11.99),
 ('King doll', 9.49),
 ('Queen doll', 9.49),
 ('12 inch teddy bear', 8.99),
 ('8 inch teddy bear', 5.99),
 ('Raggedy Ann', 4.99),
 ('Bird bean bag toy', 3.49),
 ('Fish bean bag toy', 3.49),
 ('Rabbit bean bag toy', 3.49)]


### Завдання 4: 

Зареєструйтеся на сайті  http://www.sql-ex.ru/

Виконайте завдання із блоку SELECT; з 1 по 30 завдання

Код запитів вставляйте сюди

In [107]:
"""
1:
SELECT model, speed, hd FROM pc WHERE price < 500
2:
SELECT DISTINCT maker FROM product WHERE type = 'printer'
3:
SELECT model, ram, screen FROM laptop WHERE price > 1000
4:
SELECT * from printer WHERE color = 'y'
5:
SELECT model, speed, hd from pc WHERE price < 600 AND (cd = '12x' OR cd = '24x')
6:
SELECT Maker, Speed FROM laptop 
INNER JOIN product ON laptop.model = product.model
WHERE hd >= 10
GROUP BY maker, speed

7: 
SELECT DISTINCT products.model, products.price
FROM (
    SELECT model, price FROM pc
    UNION ALL
    SELECT model, price FROM printer
    UNION ALL
    SELECT model, price FROM laptop
) as products
INNER JOIN product ON products.model = product.model
WHERE product.maker = "b"

8:
SELECT maker FROM product WHERE type = 'pc'
EXCEPT
SELECT maker FROM product WHERE type = 'laptop'

9:
SELECT maker FROM product
INNER JOIN pc ON product.model = pc.model
WHERE pc.speed >= 450
GROUP BY maker

10:
SELECT DISTINCT model, price FROM printer
WHERE price = (SELECT MAX(price) FROM printer)

11:
SELECT AVG(speed) from PC

12:
SELECT AVG(speed) FROM LAPTOP WHERE price > 1000

13:
SELECT AVG(speed) from PC
INNER JOIN product ON pc.model = product.model
WHERE maker = 'a'

14:
SELECT classes.class, ships.name, classes.country FROM SHIPS
INNER JOIN classes ON classes.class = ships.class
WHERE classes.numGuns >= 10


15: 
SELECT hd FROM PC
GROUP BY hd
HAVING COUNT(model) > 1

"""

'\n1:\nSELECT model, speed, hd FROM pc WHERE price < 500\n2:\nSELECT DISTINCT maker FROM product WHERE type = \'printer\'\n3:\nSELECT model, ram, screen FROM laptop WHERE price > 1000\n4:\nSELECT * from printer WHERE color = \'y\'\n5:\nSELECT model, speed, hd from pc WHERE price < 600 AND (cd = \'12x\' OR cd = \'24x\')\n6:\nSELECT Maker, Speed FROM laptop \nINNER JOIN product ON laptop.model = product.model\nWHERE hd >= 10\nGROUP BY maker, speed\n\n7: \nSELECT DISTINCT products.model, products.price\nFROM (\n    SELECT model, price FROM pc\n    UNION ALL\n    SELECT model, price FROM printer\n    UNION ALL\n    SELECT model, price FROM laptop\n) as products\nINNER JOIN product ON products.model = product.model\nWHERE product.maker = "b"\n\n8:\nSELECT maker FROM product WHERE type = \'pc\'\nEXCEPT\nSELECT maker FROM product WHERE type = \'laptop\'\n\n9:\nSELECT maker FROM product\nINNER JOIN pc ON product.model = pc.model\nWHERE pc.speed >= 450\nGROUP BY maker\n\n10:\nSELECT DISTINCT mo

In [109]:

"""
16:
SELECT DISTINCT p1.model, p2.model, p1.speed, p1.ram FROM PC as p1
INNER JOIN pc as p2 on p1.model <> p2.model
WHERE 
p1.speed = p2.speed
AND
p1.ram = p2.ram
AND
p1.model > p2.model

17:
SELECT DISTINCT p.type, l.model, l.speed from laptop as l
INNER JOIN product as p ON l.model = p.model
WHERE l.speed < ALL(SELECT speed FROM PC)

18:
SELECT DISTINCT c.maker, a.minPrice as price
FROM (SELECT MIN(price) as minPrice 
      FROM Printer 
      WHERE color = 'y'
      ) as a 
INNER JOIN Printer as b ON a.minPrice = b.price
INNER JOIN Product as c ON b.model = c.model
WHERE color = 'y'

19:
SELECT p.maker, AVG(screen) as average
FROM laptop as l
INNER JOIN product as p ON p.model = l.model
GROUP BY maker

20:
SELECT maker, count(model) as Count_Model FROM product
WHERE type = 'pc'
GROUP BY maker
HAVING COUNT(model) >= 3

21:
SELECT maker, MAX(pc.price) as Max_price from product as p
INNER JOIN pc ON p.model = pc.model
GROUP BY maker

22:
SELECT speed, AVG(price) as avg_price FROM pc
WHERE speed > 600
GROUP BY speed

23:
SELECT maker FROM product
INNER JOIN pc ON product.model = pc.model
WHERE pc.speed >= 750

INTERSECT

SELECT maker FROM product
INNER JOIN laptop as l ON product.model = l.model
WHERE l.speed >= 750

24:
WITH All_products AS (
SELECT price, model FROM pc
UNION ALL SELECT price, model FROM laptop
UNION ALL SELECT price, model FROM printer
)
SELECT model FROM All_products
WHERE price = (SELECT MAX(price) FROM All_products)
GROUP BY model

25:
SELECT maker FROM product as p
WHERE type = 'printer'

INTERSECT

SELECT maker FROM product as p
INNER JOIN pc ON pc.model = p.model
WHERE type = 'pc'
AND
pc.speed IN (
SELECT MAX(speed) FROM pc
WHERE ram = (SELECT MIN(ram) FROM pc)
)
AND 
pc.ram = (SELECT MIN(ram) FROM pc) 

26:
WITH prods as (
SELECT pc.price, p.maker FROM pc
INNER JOIN product as p ON p.model = pc.model
UNION ALL
SELECT l.price, p.maker FROM laptop as l
INNER JOIN product as p ON p.model = l.model
)
SELECT AVG(price) FROM PRODS
WHERE maker = 'a'

27:
WITH prods as (
SELECT maker from product
WHERE type = 'pc'

INTERSECT

SELECT maker from product
WHERE type = 'printer'
)

SELECT maker, AVG(hd) FROM product
INNER JOIN pc on product.model = pc.model
WHERE maker in (SELECT maker FROM prods)
GROUP BY maker

28:
SELECT count(maker) as qty FROM
(
SELECT maker FROM product
GROUP BY maker
HAVING count(model) = 1
) as r

29:
SELECT i.point, i.date, i.inc, o.out
FROM Income_o as i
LEFT JOIN Outcome_o as o ON i.point = o.point AND i.date = o.date
UNION
SELECT o.point, o.date, i.inc, o.out
FROM Outcome_o as o
LEFT JOIN Income_o as i ON i.point = o.point AND i.date = o.date

30:
WITH SB(point, date, out, inc) AS (
    SELECT point, date, null, inc
        FROM income
        UNION ALL
        SELECT point, date, out, null
        FROM outcome
)

SELECT point, date, SUM(out) as Outcome, SUM(inc) as Income
FROM SB
GROUP BY point, date

"""

"\n15: \nSELECT hd FROM PC\nGROUP BY hd\nHAVING COUNT(model) > 1\n\n16:\nSELECT DISTINCT p1.model, p2.model, p1.speed, p1.ram FROM PC as p1\nINNER JOIN pc as p2 on p1.model <> p2.model\nWHERE \np1.speed = p2.speed\nAND\np1.ram = p2.ram\nAND\np1.model > p2.model\n\n17:\nSELECT DISTINCT p.type, l.model, l.speed from laptop as l\nINNER JOIN product as p ON l.model = p.model\nWHERE l.speed < ALL(SELECT speed FROM PC)\n\n18:\nSELECT DISTINCT c.maker, a.minPrice as price\nFROM (SELECT MIN(price) as minPrice \n      FROM Printer \n      WHERE color = 'y'\n      ) as a \nINNER JOIN Printer as b ON a.minPrice = b.price\nINNER JOIN Product as c ON b.model = c.model\nWHERE color = 'y'\n\n19:\nSELECT p.maker, AVG(screen) as average\nFROM laptop as l\nINNER JOIN product as p ON p.model = l.model\nGROUP BY maker\n"